# работа со Spark из Python

Посчитаем среднюю корректность ответов для content_id


датасет "Kaggle Riiid Answers Correctness Prediction"
* row_id: (int64) идентификатор записи
* timestamp: (int64) время от начала взаимодействия пользователя до завершения задания
* user_id: (int32) идентификатор пользователя
* content_id: (int16) идентификатор типа взаимодействия
* content_type_id: (int8) 0 - вопрос, 1 - просмотр лекции
* task_container_id: (int16) ID набора вопросов
* user_answer: (int8) ответ пользователя. -1 для лекция
* answered_correctly: (int8) маркер правильного овтета. -1 для лекций
* prior_question_elapsed_time: (float32) время, затраченное пользователей на ответ 
после прослушивания лекции
* prior_question_had_explanation: (bool) пользователю показали правильный ответ, 
после ответа на вопросы

Данильченко Вадим

In [12]:
import pyspark

conf = (
    pyspark.SparkConf().setAppName("Test")
        .set("spark.executor.instances", "1")
        .set("spark.executor.cores", "1")
        .set("spark.submit.deployMode","client")
        .set("spark.executor.memory", "1024Mb")
)
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [13]:
riiid_df = spark.read.csv("hdfs:///user/vdanilchenko/train_10m.csv", inferSchema=True, header=True)

In [14]:
riiid_df.printSchema()

root
 |-- row_id: integer (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- content_id: integer (nullable = true)
 |-- content_type_id: integer (nullable = true)
 |-- task_container_id: integer (nullable = true)
 |-- user_answer: integer (nullable = true)
 |-- answered_correctly: integer (nullable = true)
 |-- prior_question_elapsed_time: double (nullable = true)
 |-- prior_question_had_explanation: boolean (nullable = true)



In [15]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
riiid_df

row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,115,5692,0,1,3,1,null,null
1,56943,115,5716,0,2,2,1,37000.0,false
2,118363,115,128,0,0,0,1,55000.0,false
3,131167,115,7860,0,3,0,1,19000.0,false
4,137965,115,7922,0,4,1,1,11000.0,false
5,157063,115,156,0,5,2,1,5000.0,false
6,176092,115,51,0,6,0,1,17000.0,false
7,194190,115,50,0,7,3,1,17000.0,false
8,212463,115,7896,0,8,2,1,16000.0,false
9,230983,115,7863,0,9,0,1,16000.0,false


In [19]:
from pyspark.sql import functions as f

In [21]:
%%time
(
riiid_df
    .select('content_id', 'answered_correctly')
    .filter(f.col('answered_correctly')!='-1')
    .groupBy('content_id')
    .mean('answered_correctly')
    .orderBy('avg(answered_correctly)')
    .show()
)

+----------+-----------------------+
|content_id|avg(answered_correctly)|
+----------+-----------------------+
|      7848|                    0.0|
|      7849|                    0.0|
|      3095|    0.05714285714285714|
|     10062|      0.087248322147651|
|     13038|    0.09090909090909091|
|      7639|    0.11022727272727273|
|     10924|    0.13150684931506848|
|      3125|    0.13188976377952755|
|      7374|    0.14285714285714285|
|      9220|    0.15378356387306755|
|     10095|    0.15384615384615385|
|      7487|                0.15625|
|     13481|     0.1590909090909091|
|      8697|    0.16252390057361377|
|     10239|     0.1628242074927954|
|     13341|    0.16304347826086957|
|      2063|    0.16394565589679186|
|      6531|    0.16486902927580893|
|     10901|    0.16494845360824742|
|     10061|    0.16506410256410256|
+----------+-----------------------+
only showing top 20 rows

CPU times: user 10.3 ms, sys: 2.02 ms, total: 12.3 ms
Wall time: 15.3 s


In [22]:
sc.stop()